In [1]:
# default_exp sync
# default_cls_lvl 3

In [2]:
# export
from nbdev.imports import *
from nbdev.export import *

# Converting modules to notebooks

> The functions that transforms a library back to notebooks

## Finding the way back to notebooks

We need to get the name of the object we are looking for, and then we'll try to find it in our index file.

In [3]:
#export 
def _get_property_name(p):
    "Get the name of property `p`"
    if hasattr(p, 'fget'):
        return p.fget.func.__qualname__ if hasattr(p.fget, 'func') else p.fget.__qualname__
    else: return next(iter(re.findall(r'\'(.*)\'', str(p)))).split('.')[-1]

def get_name(obj):
    "Get the name of `obj`"
    if hasattr(obj, '__name__'):       return obj.__name__
    elif getattr(obj, '_name', False): return obj._name
    elif hasattr(obj,'__origin__'):    return str(obj.__origin__).split('.')[-1] #for types
    elif type(obj)==property:          return _get_property_name(obj)
    else:                              return str(obj).split('.')[-1]

In [4]:
# export
def qual_name(obj):
    "Get the qualified name of `obj`"
    if hasattr(obj,'__qualname__'): return obj.__qualname__
    if inspect.ismethod(obj):       return f"{get_name(obj.__self__)}.{get_name(fn)}"
    return get_name(obj)

In [5]:
test_eq(get_name(in_ipython), 'in_ipython')
test_eq(get_name(DocsTestClass.test), 'test')

For properties defined using `property` or our own `add_props` helper, we approximate the name by looking at their getter functions, since we don't seem to have access to the property name itself. If everything fails (a getter cannot be found), we return the name of the object that contains the property. This suffices for `source_nb` to work.

In [6]:
#hide
class PropertyClass:
    p_lambda = property(lambda x: x)
    def some_getter(self): return 7
    p_getter = property(some_getter)

test_eq(get_name(PropertyClass.p_lambda), 'PropertyClass.<lambda>')
test_eq(get_name(PropertyClass.p_getter), 'PropertyClass.some_getter')
test_eq(get_name(PropertyClass), 'PropertyClass')

In [7]:
# export
def source_nb(func, is_name=None, return_all=False):
    "Return the name of the notebook where `func` was defined"
    is_name = is_name or isinstance(func, str)
    index = get_nbdev_module().index
    name = func if is_name else qual_name(func)
    while len(name) > 0:
        if name in index: return (name,index[name]) if return_all else index[name]
        name = '.'.join(name.split('.')[:-1])

In [8]:
test_eq(qual_name(DocsTestClass), 'DocsTestClass')
test_eq(qual_name(DocsTestClass.test), 'DocsTestClass.test')

In [9]:
# export
_re_default_nb = re.compile(r'File to edit: dev/(\S+)\s+')
_re_cell = re.compile(r'^#Cell|^#Comes from\s+(\S+), cell')

You can either pass an object or its name (by default `is_name` will look if `func` is a string or not, but you can override if there is some inconsistent behavior). 

If passed a method of a class, the function will return the notebook in which the largest part of the function was defined in case there is a monkey-matching that defines `class.method` in a different notebook than `class`. If `return_all=True`, the function will return a tuple with the name by which the function was found and the notebook.

In [11]:
test_eq(source_nb(notebook2script), '00_export.ipynb')
test_eq(source_nb(DocsTestClass), '00_export.ipynb')
test_eq(source_nb(DocsTestClass.test), '00_export.ipynb')
assert source_nb(int) is None

## Reading the library

If someone decides to change a module instead of the notebooks, the following functions help update the notebooks accordingly.

In [12]:
# export
def _split(code):
    lines = code.split('\n')
    default_nb = _re_default_nb.search(lines[0]).groups()[0]
    s,res = 1,[]
    while _re_cell.search(lines[s]) is None: s += 1
    e = s+1
    while e < len(lines):
        while e < len(lines) and _re_cell.search(lines[e]) is None: e += 1
        grps = _re_cell.search(lines[s]).groups()
        nb = grps[0] or default_nb
        content = lines[s+1:e]
        while len(content) > 1 and content[-1] == '': content = content[:-1]
        res.append((nb, '\n'.join(content)))
        s,e = e,e+1
    return res

In [13]:
#export
def _relimport2name(name, mod_name):
    if mod_name.endswith('.py'): mod_name = mod_name[:-3]
    mods = mod_name.split(os.path.sep)
    i = last_index(Config().lib_name, mods)
    mods = mods[i:]
    if name=='.': return '.'.join(mods[:-1])
    i = 0
    while name[i] == '.': i += 1
    return '.'.join(mods[:-i] + [name[i:]])

In [14]:
# export
#Catches any from .bla import something and catches .bla in group 1, the imported thing(s) in group 2.
_re_loc_import = re.compile(r'(^\s*)from (\.\S*) import (.*)$')

In [15]:
test_eq(_relimport2name('.core', 'nbdev/data.py'), 'nbdev.core')
test_eq(_relimport2name('.core', 'home/sgugger/fastai_dev/nbdev/nbdev/data.py'), 'nbdev.core')
test_eq(_relimport2name('..core', 'nbdev/vision/data.py'), 'nbdev.core')
test_eq(_relimport2name('.transform', 'nbdev/vision/data.py'), 'nbdev.vision.transform')
test_eq(_relimport2name('..notebook.core', 'nbdev/data/external.py'), 'nbdev.notebook.core')

In [16]:
#export
def _deal_loc_import(code, fname):
    def _replace(m):
        sp,mod,obj = m.groups()
        return f"{sp}from {_relimport2name(mod, fname)} import {obj}"
    return '\n'.join([_re_loc_import.sub(_replace,line) for line in code.split('\n')])

In [17]:
#hide
code = "from .core import *\nnothing to see\n  from .vision import bla1, bla2"
test_eq(_deal_loc_import(code, 'nbdev/data.py'), "from nbdev.core import *\nnothing to see\n  from nbdev.vision import bla1, bla2")

In [18]:
#export
def _script2notebook(fname, dic, silent=False):
    "Put the content of `fname` back in the notebooks it came from."
    if os.environ.get('IN_TEST',0): return  # don't export if running tests
    fname = Path(fname)
    with open(fname, encoding='utf8') as f: code = f.read()
    splits = _split(code)
    assert len(splits)==len(dic[fname]), f"Exported file from notebooks should have {len(dic[fname])} cells but has {len(splits)}."
    assert np.all([c1[0]==c2[1]] for c1,c2 in zip(splits, dic[fname]))
    splits = [(c2[0],c1[0],c1[1]) for c1,c2 in zip(splits, dic[fname])]
    nb_fnames = {Config().nbs_path/s[1] for s in splits}
    for nb_fname in nb_fnames:
        nb = read_nb(nb_fname)
        for i,f,c in splits:
            c = _deal_loc_import(c, str(fname))
            if f == nb_fname:
                l = nb['cells'][i]['source'].split('\n')[0]
                nb['cells'][i]['source'] = l + '\n' + c
        NotebookNotary().sign(nb)
        nbformat.write(nb, str(nb_fname), version=4)
    
    if not silent: print(f"Converted {fname.relative_to(Config().lib_path)}.")

In [20]:
dic = notebook2script(silent=True, to_dict=True)
_script2notebook(Config().lib_path/'export.py', dic)

Converted export.py.


In [21]:
#export
def script2notebook(fname=None, silent=False):
    if os.environ.get('IN_TEST',0): return
    dic = notebook2script(silent=True, to_dict=True)
    exported = get_nbdev_module().modules
    
    if fname is None: 
        files = [f for f in Config().lib_path.glob('**/*.py') if str(f.relative_to(Config().lib_path)) in exported]
    else: files = glob.glob(fname)
    [ _script2notebook(f, dic, silent=silent) for f in files]

In [25]:
script2notebook()

Converted showdoc.py.
Converted test.py.
Converted sync.py.
Converted export.py.
Converted export2html.py.


## Diff notebook - library

In [26]:
#export
import subprocess
from distutils.dir_util import copy_tree

In [27]:
#export
def diff_nb_script():
    "Print the diff between the notebooks and the library in `lib_folder`"
    lib_folder = Config().lib_path
    with tempfile.TemporaryDirectory() as d1, tempfile.TemporaryDirectory() as d2:
        copy_tree(Config().lib_path, d1)
        notebook2script(silent=True)
        copy_tree(Config().lib_path, d2)
        shutil.rmtree(Config().lib_path)
        shutil.copytree(d1, str(Config().lib_path))
        for d in [d1, d2]:
            if (Path(d)/'__pycache__').exists(): shutil.rmtree(Path(d)/'__pycache__')
        res = subprocess.run(['diff', '-ru', d1, d2], stdout=subprocess.PIPE)
        print(res.stdout.decode('utf-8'))

In [30]:
diff_nb_script()

## Export

In [31]:
#hide
notebook2script()

Converted 01_sync.ipynb.
Converted 03_export2html.ipynb.
Converted 04_test.ipynb.
Converted 09_index.ipynb.
Converted 02_showdoc.ipynb.
Converted 00_export.ipynb.
